<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=17c7d47c9f34dc317d1e2325b1dc5a6e3fadacddb48bb77247d151c6b02939ca
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 12:13:40
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 79.97 K (0.59%)
Current PnL: -20.06 L (-13.49%)
CY Booked + Current PnL: -11.02 L (-7.41%)
-------------------
Total profit:  2.79 L
Total loss:  -22.85 L
-------------------
Total Booked + Current PnL: 15.96 L (13.11%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.59%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.44 L (59.37%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.64,112904.0,8592.0,8061.0,0.61,8.24,7.14,15.96,9.0,1.07,0.85,26.53,X5K,ATH,METALS
53,MEDANTA,1486.00,13.45,61.0,H-MC,15.07,162040.0,35840.0,10338.0,-0.21,28.40,6.38,36.59,108.0,3.47,1.22,44.52,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-15.74,54.0,H-LC,8.39,168300.0,12123.0,13969.0,-1.01,7.76,8.30,16.71,22.0,0.87,1.27,26.55,X40N,BTT,HEALTHCARE
81,TTKPRESTIG,770.00,99.79,60.0,M-SC,8.17,86060.0,-14717.0,14811.0,1.51,-14.60,17.21,0.09,245.0,-0.99,0.65,13.06,OX40N,NTT,DURABLES
66,SBIN,863.00,-14.20,43.0,M-LC,4.27,213391.0,11912.0,15300.0,0.41,5.91,7.17,13.51,60.0,0.78,1.61,19.25,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.41,40.0,H-SC,9.96,102450.0,-50.0,40878.0,1.05,-0.05,39.90,39.83,114.0,-0.00,0.77,1.44,AR,ATH,MISC
39,ICICIPRULI,790.00,-20.31,38.0,H-MC,2.36,137035.0,1247.0,41508.0,-0.53,0.92,30.29,31.48,107.0,0.03,1.03,13.15,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.11,37.0,H-MC,4.91,137218.0,1085.0,30037.0,-0.32,0.80,21.89,22.86,91.0,0.04,1.03,13.51,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.01,37.0,H-LC,13.45,242568.0,-19051.0,138264.0,1.06,-7.28,57.00,45.57,7.0,-0.14,1.83,3.21,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,0.22,39.0,H-LC,10.20,228937.0,-12987.0,76259.0,0.58,-5.37,33.31,26.15,8.0,-0.17,1.72,8.19,X40,ATH,IT
47,JIOFIN,387.00,-0.75,42.0,H-LC,15.26,219520.0,2114.0,51390.0,1.13,0.97,23.41,24.61,48.0,0.04,1.65,56.32,XY24,BTT,FINANCE
87,VBL,671.64,-12.51,44.0,H-LC,8.05,314853.0,-989.0,113662.0,-0.84,-0.31,36.10,35.67,5.0,-0.01,2.37,13.50,X40N,ATH,FMCG
84,UNITDSPR,1644.00,-14.23,45.0,H-LC,8.70,172643.0,-9116.0,44369.0,2.06,-5.02,25.70,19.39,86.0,-0.21,1.30,2.72,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-11.91,52.0,H-LC,2.82,271206.0,5780.0,51014.0,-0.29,2.18,18.81,21.40,11.0,0.11,2.04,9.67,XY25,NTT,FMCG
51,LTIM,7230.2,-6.94,48.0,H-LC,2.92,231008.0,-15143.0,94344.0,0.21,-6.15,40.84,32.18,16.0,-0.16,1.74,27.67,X200,ATH,IT
45,ITC,452.0,-38.34,46.0,H-LC,3.02,198803.0,-1335.0,22226.0,1.41,-0.67,11.18,10.44,4.0,-0.06,1.50,4.86,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,62.0,H-LC,4.41,217282.0,12782.0,28638.0,0.36,6.25,13.18,20.25,10.0,0.45,1.63,15.53,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.41,57.0,H-LC,7.22,221118.0,-30650.0,152881.0,0.95,-12.17,69.14,48.55,27.0,-0.20,1.66,18.94,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,72.0,L-SC,26.78,89664.0,-23885.0,63939.0,9.78,-21.03,71.31,35.27,268.0,-0.37,0.67,119.45,XR,NTT,HOTELS
60,RELAXO,1176.00,-34.17,57.0,H-SC,8.53,61107.0,-54431.0,89418.0,6.63,-47.11,146.33,30.28,136.0,-0.61,0.46,20.14,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-164.77,53.0,M-SC,8.22,74814.0,-27396.0,120787.0,5.70,-26.80,161.45,91.37,208.0,-0.23,0.56,21.45,XY25,NTT,FINANCE
23,COLPAL,3767.14,0.50,58.0,H-MC,10.58,183121.0,-28620.0,110715.0,3.76,-13.52,60.46,38.77,84.0,-0.26,1.38,8.99,XY25,ATH,FMCG
12,BATAINDIA,2096.00,-38.58,42.0,M-SC,9.60,91790.0,-36880.0,82180.0,2.46,-28.66,89.53,35.20,219.0,-0.45,0.69,5.35,X40,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,6550.00,42.0,L-SC,5.74,72961.0,-20849.0,97469.0,-1.46,-22.22,133.59,81.67,269.0,-0.21,0.55,41.29,XR,NTT,CERAMICS
25,DEN,75.0,102.07,40.0,M-SC,2.47,56480.0,-27984.0,63517.0,-1.31,-33.13,112.46,42.07,237.0,-0.44,0.42,19.95,AR,NTT,ENTERTAINMENT
42,INDUSINDBK,1800.0,246.64,33.0,L-MC,10.85,42556.0,-34650.0,60042.0,-1.30,-44.88,141.09,32.89,258.0,-0.58,0.32,17.20,XR,NTT,BANKS
43,INFY,2275.0,-18.88,46.0,H-LC,11.72,315453.0,2565.0,169114.0,-1.27,0.82,53.61,54.87,3.0,0.02,2.37,7.52,X40,BTT,IT
85,VAIBHAVGBL,521.0,35.54,35.0,H-SC,12.09,90541.0,-61450.0,135576.0,-1.24,-40.43,149.74,48.77,125.0,-0.45,0.68,5.38,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,99.79,60.0,M-SC,8.17,86060.0,-14717.0,14811.0,1.51,-14.60,17.21,0.09,245.0,-0.99,0.65,13.06,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,41.0,H-SC,14.61,115209.0,-27270.0,46786.0,0.57,-19.14,40.61,13.70,163.0,-0.58,0.87,35.98,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.52,51.0,H-SC,3.45,216648.0,-53019.0,89346.0,1.13,-19.66,41.24,13.47,138.0,-0.59,1.63,9.58,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,42.0,H-SC,3.02,113580.0,-30223.0,56972.0,1.01,-21.02,50.16,18.60,149.0,-0.53,0.85,25.34,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,108.15,39.0,M-SC,2.19,136350.0,-38209.0,38246.0,0.21,-21.89,28.05,0.02,221.0,-1.00,1.03,18.60,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-27.08,48.0,H-LC,15.59,290798.0,-55158.0,179015.0,-0.00,-15.94,61.56,35.80,1.0,-0.31,2.19,3.02,X40,BTT,IT
43,INFY,2275.00,-18.88,46.0,H-LC,11.72,315453.0,2565.0,169114.0,-1.27,0.82,53.61,54.87,3.0,0.02,2.37,7.52,X40,BTT,IT
45,ITC,452.00,-38.34,46.0,H-LC,3.02,198803.0,-1335.0,22226.0,1.41,-0.67,11.18,10.44,4.0,-0.06,1.50,4.86,X40,NTT,FMCG
87,VBL,671.64,-12.51,44.0,H-LC,8.05,314853.0,-989.0,113662.0,-0.84,-0.31,36.10,35.67,5.0,-0.01,2.37,13.50,X40N,ATH,FMCG
1,ABB,7934.00,-42.01,37.0,H-LC,13.45,242568.0,-19051.0,138264.0,1.06,-7.28,57.00,45.57,7.0,-0.14,1.83,3.21,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.45,61.0,H-MC,15.07,162040.0,35840.0,10338.0,-0.21,28.40,6.38,36.59,108.0,3.47,1.22,44.52,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.62,74.0,H-LC,12.84,233284.0,48803.0,21555.0,1.30,26.45,9.24,38.14,38.0,2.26,1.76,39.86,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,61.0,H-MC,15.07,162040.0,35840.0,10338.0,-0.21,28.40,6.38,36.59,108.0,3.47,1.22,44.52,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.78,60.0,H-MC,4.04,189480.0,24810.0,43164.0,0.78,15.07,22.78,41.28,89.0,0.57,1.43,24.25,X40N,ATH,AC
88,VOLTAS,1918.49,-2.06,58.0,H-MC,6.35,205500.0,13758.0,82282.0,0.79,7.18,40.04,50.08,99.0,0.17,1.55,14.46,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.87,178378.0,27465.0,29843.0,0.09,18.20,16.73,37.97,93.0,0.92,1.34,48.11,AR,ATH,AUTO
24,DABUR,735.00,-6.72,53.0,H-MC,3.78,203054.0,6026.0,83597.0,1.32,3.06,41.17,45.49,102.0,0.07,1.53,16.50,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.87,178378.0,27465.0,29843.0,0.09,18.20,16.73,37.97,93.0,0.92,1.34,48.11,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,51.0,H-SC,12.48,137742.0,14406.0,119353.0,0.82,11.68,86.65,108.45,119.0,0.12,1.04,36.64,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-41.93,50.0,M-SC,5.60,98250.0,6752.0,71998.0,1.03,7.38,73.28,86.07,223.0,0.09,0.74,42.68,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,51.0,M-LC,7.71,155180.0,4235.0,105228.0,-0.14,2.81,67.81,72.51,53.0,0.04,1.17,9.00,XR,NTT,IT
59,RAJOOENG,143.33,-32.41,40.0,H-SC,9.96,102450.0,-50.0,40878.0,1.05,-0.05,39.90,39.83,114.0,-0.00,0.77,1.44,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.36,37.0,H-SC,16.52,87204.0,-13759.0,108290.0,0.21,-13.63,124.18,93.63,148.0,-0.13,0.66,29.89,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,72.0,L-SC,26.78,89664.0,-23885.0,63939.0,9.78,-21.03,71.31,35.27,268.0,-0.37,0.67,119.45,XR,NTT,HOTELS
52,MASFIN,398.61,-19.15,47.0,H-SC,12.72,91965.0,-6015.0,27617.0,0.00,-6.14,30.03,22.05,152.0,-0.22,0.69,33.68,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.64,112904.0,8592.0,8061.0,0.61,8.24,7.14,15.96,9.0,1.07,0.85,26.53,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,41.0,H-SC,14.61,115209.0,-27270.0,46786.0,0.57,-19.14,40.61,13.70,163.0,-0.58,0.87,35.98,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,72.0,L-SC,26.78,89664.0,-23885.0,63939.0,9.78,-21.03,71.31,35.27,268.0,-0.37,0.67,119.45,XR,NTT,HOTELS
27,DMART,5201.00,-6.62,74.0,H-LC,12.84,233284.0,48803.0,21555.0,1.30,26.45,9.24,38.14,38.0,2.26,1.76,39.86,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.87,178378.0,27465.0,29843.0,0.09,18.20,16.73,37.97,93.0,0.92,1.34,48.11,AR,ATH,AUTO
14,BLUESTARCO,2326.38,9.78,60.0,H-MC,4.04,189480.0,24810.0,43164.0,0.78,15.07,22.78,41.28,89.0,0.57,1.43,24.25,X40N,ATH,AC
23,COLPAL,3767.14,0.50,58.0,H-MC,10.58,183121.0,-28620.0,110715.0,3.76,-13.52,60.46,38.77,84.0,-0.26,1.38,8.99,XY25,ATH,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.67
1,25,43.09
2,50,72.73


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.04
LC    36.78
MC    26.18
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.17
X40      15.47
X40N     14.27
XY25      9.77
AR        9.33
XR        8.94
OX40N     7.32
X5K       2.24
X200      1.74
MH        1.67
SR        1.08
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.62
H-MC    23.32
H-SC    22.32
M-SC    13.15
M-LC     7.11
M-MC     2.54
L-SC     1.57
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-1.12,34.06
IT,13.35,-16.40,74.13
BANKS,7.34,-17.02,66.71
FINANCE,7.21,-25.58,71.89
PAINTS,5.94,-17.03,43.22
MISC,5.84,-18.37,86.22
ELECTRICAL,5.66,-9.58,69.98
AUTO,4.55,-11.71,60.31
AC,3.71,1.65,31.65


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2893068.0,23
AR,1240659.0,10
XR,1104083.0,13
X40,910759.0,10
X40N,538198.0,11
OX40N,485631.0,10
XY25,443582.0,7
SR,200007.0,2
X5K,140688.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2808629.0,27
M-SC,1497146.0,17
H-LC,1427664.0,17
H-MC,1280080.0,18
M-LC,434656.0,5
M-MC,339374.0,2
L-SC,251775.0,3
L-MC,60042.0,1
L-LC,45035.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,946000.0,6
M-SC,XY24,880118.0,7
H-SC,AR,667091.0,5
H-LC,X40,599495.0,5
H-SC,XR,534163.0,6
H-MC,XY24,494685.0,5
H-LC,AR,382718.0,2
M-MC,XY24,339374.0,2
H-MC,X40,229084.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
